In [1]:
import sys
from pathlib import Path
import os

# ---------------------------------------------------------
# 1) path assoluto della directory dove sta questo script
HERE = Path.cwd()  # Use the current working directory in Jupyter Notebook

# 2) path assoluto della root del progetto (un livello sopra "scripts")
PROJECT_ROOT = HERE.parent

# 3) inserisco la root all’inizio di sys.path
sys.path.insert(0, str(PROJECT_ROOT))

In [17]:
import src.pipeline.Real_Pipeline as rp
import src.tensors.cokurt as ck
import tensorflow as tf

In [ ]:
params = {}

In [21]:
n_stocks = 50

In [ ]:
dataset = rp.real_data_pipeline(batch_size=1, n_days=1200, n_days_out=5, n_stocks=n_stocks, n_days_range=[250,250],
                      filename_returns ="../DATA/vanilla_returns_crsp-90-24.joblib",
                      filename_available_stocks ='../DATA/dtin_1200_dtout_5_shift_1_Db_crsp-90-24.pkl',
                      datatime_range=['1995-01-01','2001-01-01'], return_generator=True, oos_covariance=False)

In [27]:
next(dataset())

(array([[[ 0.008499, -0.019663, -0.005731, ...,  0.      ,  0.      ,
          -0.024869],
         [-0.014925,  0.002165, -0.028078, ..., -0.00545 ,  0.008219,
          -0.024457],
         [ 0.00213 , -0.012752, -0.006459, ...,  0.      , -0.036613,
           0.010689],
         ...,
         [ 0.033573, -0.032483, -0.007194, ...,  0.016878,  0.033195,
          -0.038153],
         [ 0.01986 ,  0.004582, -0.014823, ...,  0.007587,  0.013554,
           0.002972],
         [ 0.004902, -0.009756, -0.004926, ...,  0.038889,  0.02139 ,
          -0.020942]]]),
 array([[[-0.009396, -0.00813 , -0.004098, -0.001372, -0.004121],
         [-0.013928,  0.028249,  0.038462,  0.010582,  0.      ],
         [-0.042303,  0.065031, -0.018433,  0.015258,  0.05896 ],
         [-0.01766 ,  0.003371, -0.011198,  0.001133,  0.016968],
         [ 0.013459,  0.      , -0.002656,  0.014647, -0.002625],
         [-0.023881,  0.022936, -0.026906, -0.015361,  0.065523],
         [-0.014105,  0.00148 , -0.

In [ ]:
ccore = tf.zeros((n_stocks, n_stocks, n_stocks, n_stocks), dtype=tf.float32)

cnt = 0
for rin,rout in dataset():
    if tf.reduce_any(tf.math.reduce_std(rout, axis=-1)==0):
        # Se ci sono colonne costanti, gestisci il caso
        continue
    rin = tf.constant(rin[0], dtype=tf.float32)
    rout = tf.constant(rout[0], dtype=tf.float32)
    
    cnt += 1
    K = ck.cokurtosi_tensore_tf(rin, standardizza=True, dtype=tf.float32)
    G, U = ck.tucker_simmetrico_tf(K, r=None, dtype=tf.float32, return_fattori=True)
    Cout = ck.cokurtosi_tensore_tf(rout, standardizza=True, dtype=tf.float32)
    ccore += ck.core_da_fattori_tf(Cout, U)
    print(cnt,end='\r')
    if cnt>100000:
        break

ccore /= cnt


KeyboardInterrupt: 

In [29]:
dataset = rp.real_data_pipeline(batch_size=252, n_days=1200, n_days_out=1, n_stocks=n_stocks, n_days_range=[250,250], sequential=True,
                      filename_returns ="../DATA/vanilla_returns_crsp-90-24.joblib",
                      filename_available_stocks ='../DATA/dtin_1200_dtout_5_shift_1_Db_crsp-90-24.pkl',
                      datatime_range=['1995-01-01','2001-01-01'], return_generator=True, oos_covariance=False)

In [30]:
rin,rout = next(dataset())

In [31]:
rin.shape

(252, 50, 250)